In [2]:
!pip install streamlit pyngrok --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.6 MB/s eta 0:00:00


In [ ]:
from pyngrok import ngrok
!streamlit run apphhh.py &>/dev/null&  # Run the Streamlit app in the background
public_url = ngrok.connect(8501, "http")
print(f"Streamlit app is live at {public_url}")

Streamlit app is live at NgrokTunnel: "https://fedd-34-127-125-111.ngrok-free.app" -> "http://localhost:8501"


In [1]:
from pyngrok import ngrok

for tunnel in ngrok.get_tunnels():
    ngrok.disconnect(tunnel.public_url)

!killall streamlit

!streamlit run apph.py &>/dev/null&
public_url = ngrok.connect(8501, "http")
print(f"Streamlit app is live at {public_url}")

ModuleNotFoundError: No module named 'pyngrok'

In [ ]:
!pip install pyngrok --quiet
from pyngrok import ngrok

ngrok.set_auth_token("2pL9PbQIhS5E4gjYRa3NPBuYVJg_CDReKu48Yq7nLUHNpuFD")

for tunnel in ngrok.get_tunnels():
    ngrok.disconnect(tunnel.public_url)

!killall streamlit

!streamlit run apph.py &>/dev/null&
public_url = ngrok.connect(8501, "http")
print(f"Streamlit app is live at {public_url}")

streamlit: no process found
Streamlit app is live at NgrokTunnel: "https://597d-34-127-125-111.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
%%writefile apph.py
import streamlit as st
import tensorflow as tf
from PIL import Image
import numpy as np
import plotly.express as px

model = tf.keras.models.load_model("/content/model.keras")

def preprocess_image(image):
    """Resize and normalize the image for the model."""
    image = image.resize((299, 299))
    image_array = np.array(image)
    if len(image_array.shape) == 2:
        image_array = np.stack([image_array] * 3, axis=-1)
    image_array = image_array / 255.0
    return np.expand_dims(image_array, axis=0)

st.set_page_config(
    page_title="Brain MRI Classification",
    page_icon="🧠",
    layout="wide",
)

st.markdown(
    """
    <style>
    .main-title {
        font-size: 36px;
        font-weight: bold;
        color: #4CAF50;
        text-align: center;
    }
    .sub-title {
        font-size: 18px;
        color: #555;
        text-align: center;
    }
    </style>
    """,
    unsafe_allow_html=True,
)
st.markdown('<div class="main-title">🧠 Brain MRI Tumor Classifier</div>', unsafe_allow_html=True)
st.markdown('<div class="sub-title">Upload an MRI image to classify tumor type with high accuracy</div>', unsafe_allow_html=True)

uploaded_file = st.file_uploader("Upload your MRI image (JPG, PNG)", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption="Uploaded MRI Image", use_column_width=True)

    progress = st.progress(0)
    for percent_complete in range(0, 101, 10):
        progress.progress(percent_complete)

    processed_image = preprocess_image(image)
    predictions = model.predict(processed_image)
    predicted_class_idx = np.argmax(predictions, axis=1)[0]
    confidence = np.max(predictions) * 100

    class_labels = [
        "Astrocitoma",
        "Carcinoma",
        "Ependimoma",
        "Papilloma",
        "Schwannoma",
        "Tuberculoma",
        "Glioma",
        "Meningioma",
        "Not Tumor",
        "Pituitary",
    ]
    predicted_class_label = class_labels[predicted_class_idx]

    st.success(f"Prediction: {predicted_class_label}")
    st.info(f"Confidence: {confidence:.2f}%")

    st.markdown("### Prediction Probability")
    fig = px.bar(
        x=class_labels,
        y=predictions.flatten() * 100,
        labels={"x": "Tumor Type", "y": "Confidence (%)"},
        color=predictions.flatten() * 100,
        color_continuous_scale="RdBu",
    )
    st.plotly_chart(fig, use_container_width=True)

st.markdown("---")
st.markdown(
    """

    """,
    unsafe_allow_html=True,
)

Writing apph.py
